In [29]:
%pip install -r requirements.txt
%pip install redis

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [30]:
# database stuff
!gdown --id 1tHWB6u3yQCuAgOYc-DxtZ8Mru3uV5_lj

/home/nk/.miniconda3/lib/python3.11/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1tHWB6u3yQCuAgOYc-DxtZ8Mru3uV5_lj
From (redirected): https://drive.google.com/uc?id=1tHWB6u3yQCuAgOYc-DxtZ8Mru3uV5_lj&confirm=t&uuid=86955f2f-2d6c-40c4-bfe6-f68e76b04304
To: /home/nk/hackaton-practice/ecomerce-with-chatbot/product_data.csv
100%|████████████████████████████████████████| 225M/225M [00:03<00:00, 71.4MB/s]


In [31]:
from dotenv import load_dotenv

load_dotenv()

True

## Preprocess dataset

In [32]:
import pandas as pd
 
MAX_TEXT_LENGTH=1000  # Maximum num of text characters to use
 
def auto_truncate(val):

    """Truncate the given text."""
    return val[:MAX_TEXT_LENGTH]
 
# Load Product data and truncate long text fields
 
all_prods_df = pd.read_csv("product_data.csv", converters={
 
    'bullet_point': auto_truncate,
    'item_keywords': auto_truncate,
    'item_name': auto_truncate
})

# Replace empty strings with None and drop
 
all_prods_df['item_keywords'].replace('', None, inplace=True)
# Drop na from every column
all_prods_df.dropna(inplace=True)

# Reset pandas dataframe index
all_prods_df.reset_index(drop=True, inplace=True)

all_prods_df.head()

,item_id,marketplace,country,main_image_id,domain_name,bullet_point,item_keywords,material,brand,color,item_name,model_name,model_number,product_type
0,B07T2JY31Y,Amazon,IN,71vX7qIEAIL,amazon.in,3D Printed Hard Back Case Mobile Cover for Son...,mobile cover back cover mobile case phone case...,Wood,Amazon Brand - Solimo,others,Amazon Brand - Solimo Designer Leaf on Wood 3D...,Sony Xperia Z1 L39H,gz8056-SL40528,CELLULAR_PHONE_CASE
1,B0854774X5,Amazon,IN,81xaJCVnl3L,amazon.in,"Snug fit for Nokia 8.1, with perfect cut-outs ...",Back Cover Designer Case Designer Take It Easy...,Silicon,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Designer Take It Easy UV...,Nokia 8.1,UV10714-SL40617,CELLULAR_PHONE_CASE
2,B085494J1X,Amazon,IN,71iB+TUHVyL,amazon.in,"Snug fit for Lenovo Z6 Pro, with perfect cut-o...",Back Cover Designer Case Designer Panda Textur...,Silicon,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Designer Panda Texture U...,Lenovo Z6 Pro,UV10833-SL40408,CELLULAR_PHONE_CASE
3,B08511FPLP,Amazon,IN,81Gd4qnyogL,amazon.in,"Snug fit for Samsung Galaxy M31, with perfect ...",Samsung Galaxy M31 mobile case Samsung Galaxy ...,Plastic,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Designer Old Stambh 3D P...,Samsung Galaxy M31,gz8759-SL40523,CELLULAR_PHONE_CASE
4,B08542LQNW,Amazon,IN,719JbIZ9gqL,amazon.in,"Snug fit for Huawei Y9 (2019), with perfect cu...",Back Cover Designer Case Designer Happy Mobile...,Silicon,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Designer Happy UV Printe...,Huawei Y9 (2019),UV10721-SL13243,CELLULAR_PHONE_CASE


In [33]:
# Num products to use (subset)
NUMBER_PRODUCTS = 2500  

# Get the first 1000 products with non-empty item keywords
product_metadata = ( 
    all_prods_df
     .head(NUMBER_PRODUCTS)
     .to_dict(orient='index')
)

# Check one of the products
product_metadata[0]

{'item_id': 'B07T2JY31Y',
 'marketplace': 'Amazon',
 'country': 'IN',
 'main_image_id': '71vX7qIEAIL',
 'domain_name': 'amazon.in',
 'bullet_point': '3D Printed Hard Back Case Mobile Cover for Sony Xperia Z1 L39H Easy to put & take off with perfect cutouts for volume buttons, audio & charging ports. Stylish design and appearance, express your unique personality. Extreme precision design allows easy access to all buttons and ports while featuring raised bezel to life screen and camera off flat surface. Slim Hard Back Cover No Warranty',
 'item_keywords': 'mobile cover back cover mobile case phone case mobile panel phone panel LG mobile case LG phone cover LG back case hard case 3D printed mobile cover mobile cover back cover mobile case phone case mobile panel phone panel Sony Xperia mobile case Sony Xperia phone cover Sony Xperia back case hard case 3D printed mobile cover mobile cover back cover mobile case phone case mobile panel phone panel Sony Xperia mobile case Sony Xperia phone 

## Setup redis as vector DB

In [34]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.redis import Redis as RedisVectorStore

# data that will be embedded and converted to vectors
texts = [
    v['item_name'] for k, v in product_metadata.items()
]

# product metadata that we'll store along our vectors
metadatas = list(product_metadata.values())

# we will use OpenAI as our embeddings provider
embedding = OpenAIEmbeddings(
            deployment="educai_emb",
            model="text-embedding-ada-002",
)

# name of the Redis search index to create
index_name = "products"

# assumes you have a redis stack server running on within your docker compose network
redis_url = "redis://localhost:6379"

# create and load redis with documents
vectorstore = RedisVectorStore.from_texts(
    texts=texts,
    metadatas=metadatas,
    embedding=embedding,
    index_name=index_name,
    redis_url=redis_url
)

## Redis as History Manager

In [35]:
import redis

# Conectar a la base de datos de Redis.
# Asegúrate de que Redis esté ejecutándose y de que los detalles de conexión sean correctos.
db = redis.StrictRedis(host='localhost', port=6379, db=0)

def add_message_to_history(user_id, message):
    key = f"chat_history:{user_id}"
    db.rpush(key, message)

def get_message_history(user_id):

    key = f"chat_history:{user_id}"
    return db.lrange(key, 0, -1)

## Chat

In [38]:
import logging
from langchain.chat_models import AzureChatOpenAI
from langchain.callbacks.base import BaseCallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts.prompt import PromptTemplate

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

#Generate UUID for user
import uuid

user_id = str(uuid.uuid4())


template = """Dado el siguiente historial de chat y una pregunta de seguimiento, reformule la pregunta de entrada de seguimiento para que sea una pregunta independiente.
O finalice la conversación si parece que ha terminado.

Historial del Chat:\"""
{chat_history}
\"""

La entrada: \"""
{question}
\"""

Standalone question:"""
condense_question_prompt = PromptTemplate.from_template(template)

template = """Eres un asistente de compras minorista amigable y conversacional. Utilice el siguiente contexto, incluidos nombres de productos, descripciones y palabras clave, para mostrarle al comprador lo que está disponible, ayudarlo a encontrar lo que busca y responder cualquier pregunta.
Está bien si no sabes la respuesta.

Contexto:\"""
{context}
\"""

Pregunta:\"
\"""

Helpful Answer:"""
qa_prompt= PromptTemplate.from_template(template)

try:
    llm = AzureChatOpenAI(
        deployment_name="educai_chat35",
        model_name="gpt-35-turbo",
    )
    
    streaming_llm = AzureChatOpenAI(
        streaming=True,
        callback_manager=BaseCallbackManager([
            StreamingStdOutCallbackHandler()]),
        verbose=True,
        deployment_name="educai_chat35",
        model_name="gpt-35-turbo",
    )
    
    question_generator = LLMChain(
        llm=llm,
        prompt=condense_question_prompt
    )
    
    doc_chain = load_qa_chain(
        llm=streaming_llm,
        chain_type="stuff",
        prompt=qa_prompt
    )
    
    chatbot = ConversationalRetrievalChain(
        retriever=vectorstore.as_retriever(),
        combine_docs_chain=doc_chain,
        question_generator=question_generator
    )

    chat_history = get_message_history(user_id)
    question = input("Hola! ¿Qué buscas hoy?")
    
    while True:
        print(f"----------------------------\nUser {user_id} asked: {question}\n----------------------------\n")

        result = chatbot(
            {"question": question, "chat_history": chat_history}
        )

        print(f"----------------------------\nBot responded to {user_id} with: {result['answer']}\n----------------------------\n")

        add_message_to_history(user_id, f"User: {question}")
        add_message_to_history(user_id, f"Bot: {result['answer']}")

        chat_history.append((result["question"], result["answer"]))
        question = input()

except Exception as e:
    print(f"An error occurred: {e}")


2023-10-01 13:35:53,424 - INFO - User e037b980-725c-49ea-9190-3986b473471d asked: Busco cosas para el baño
2023-10-01 13:35:54,132 - WARNING - score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.


¡Hola! Bienvenido/a a nuestra tienda. ¿En qué puedo ayudarte hoy? ¿Buscas algo en particular? Podemos ofrecerte el soporte para papel higiénico moderno con resorte de AmazonBasics AB-BR809-OR en acabado bronce frotado con aceite, o también el contenedor de almacenamiento de tela de AmazonBasics en un diseño alto y redondo en color natural. Si tienes alguna pregunta específica sobre estos productos, estaré encantado/a de ayudarte.Bot responded to e037b980-725c-49ea-9190-3986b473471d with: ¡Hola! Bienvenido/a a nuestra tienda. ¿En qué puedo ayudarte hoy? ¿Buscas algo en particular? Podemos ofrecerte el soporte para papel higiénico moderno con resorte de AmazonBasics AB-BR809-OR en acabado bronce frotado con aceite, o también el contenedor de almacenamiento de tela de AmazonBasics en un diseño alto y redondo en color natural. Si tienes alguna pregunta específica sobre estos productos, estaré encantado/a de ayudarte.


2023-10-01 13:36:40,745 - INFO - User e037b980-725c-49ea-9190-3986b473471d asked: Puedes darme detalles del soporte para el papel?
2023-10-01 13:36:42,007 - WARNING - score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.


¡Hola! Bienvenido/a a nuestro asistente de compras minorista. ¿En qué puedo ayudarte hoy? ¿Estás interesado/a en comprar algunos accesorios para el hogar de AmazonBasics? Si es así, puedo ofrecerte el soporte para papel higiénico moderno con resorte AB-BR809-OR, en color bronce frotado con aceite. También tenemos las bisagras redondeadas de 3.5 pulgadas x 3.5 pulgadas, también en bronce frotado con aceite, en un paquete de 18 unidades. ¿Te gustaría saber más sobre estos productos o tienes alguna otra pregunta? Estoy aquí para ayudarte.Bot responded to e037b980-725c-49ea-9190-3986b473471d with: ¡Hola! Bienvenido/a a nuestro asistente de compras minorista. ¿En qué puedo ayudarte hoy? ¿Estás interesado/a en comprar algunos accesorios para el hogar de AmazonBasics? Si es así, puedo ofrecerte el soporte para papel higiénico moderno con resorte AB-BR809-OR, en color bronce frotado con aceite. También tenemos las bisagras redondeadas de 3.5 pulgadas x 3.5 pulgadas, también en bronce frotado c

KeyboardInterrupt: Interrupted by user